<span class="girk"><출처></span>

- https://medium.com/daangn/pytorch-multi-gpu-%ED%95%99%EC%8A%B5-%EC%A0%9C%EB%8C%80%EB%A1%9C-%ED%95%98%EA%B8%B0-27270617936b

# 1. Single GPU

In [8]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)
# model = Model()
# model.to(device)

cuda:2


# 2. Multi GPU 

![image](https://user-images.githubusercontent.com/48466625/63988848-0cbf5f00-cb19-11e9-821a-2d535cf92a07.png)

- PyTorch에서는 기본적으로 multi-gpu 학습을 위한 ```Data Parallel``` 기능을 제공
- 우선 model을 각 GPU에 복사해서 할당해야 하고, iteration마다 batch를 GPU 갯수만큼 나눈다.(이 과정을 __scatter__)
- 이렇게 입력을 나누고나면, 각 GPU에서 forward 과정이 진행되고,
- 각 입력에 대해 모델이 출력을 내보내면 이제 이 출력들을 하나의 GPU로 모은다.(__gather__)
- Back propagation은 각 GPU에서 수행해서, 각 GPU에 있던 모델의 gradient를 구하고, 또 하나의 GPU로 모아서 업데이트 진행

In [ ]:
import torch
import torch.nn as nn

#### nn.DataParallel로 model을 감싸주면, 
# 위 언급한대로 replicate -> scatter -> parallel_apply -> gather 순서대로 진행함

model = nn.DataParallel(model)

#### RNN 계열 사용할떄는
### nn.DataPArallel(model, dim=1).to(device) 해야 잘된다.

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model.to(device)

<span class="mark">__number of workers : recommended to set (4*num_GPUs)__</span>

## 2.1. 메모리 불균형 문제 해결?

![image](https://user-images.githubusercontent.com/48466625/63991361-68dab100-cb22-11e9-8007-f4055bc07758.png)

- 0번이 다른 GPU에 비해 6기가정도 더 많은 메모리를 사용하고 있음. 
  - 이렇게 하나의 GPU가 상대적으로 많은 메모리를 사용하면, batch size를 많이 키울 수 없다.
  - <span class="mark">메모리 사용의 불균형을 해결해야함.</span>

In [ ]:
model = BERT(args)
model = torch.nn.DataParallel(model)
model.cuda()

...

for i, (inputs, labels) in enumerate(trainloader):
    outputs = model(inputs)          
    loss = criterion(outputs, labels)     
    
    optimizer.zero_grad()
    loss.backward()                        
    optimizer.step()

- 메모리 불균형을 간단히 해결하는 방법은, 출력을 다른 GPU로 모으는 것.
  - Default로 설정된 GPU의 경우 gradient 또한 이 GPU로 모이기에 메모리 사용량이 상당히 많음
- 간단히 출력을 모으고 싶은 GPU 번호 ```output_device``` 를 설정하면 됨.

In [ ]:
import os
import torch.nn as nn

os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2, 3'
model = nn.DataParallel(model, output_device=1)

![image](https://user-images.githubusercontent.com/48466625/63991478-06ce7b80-cb23-11e9-8cd0-6668fb650661.png)

- <span class="mark">그러나, 이는 적절한 해결방법이 아님.</span>
  - 1번 메모리가 늘었지만, 여전히 균형이 잡히지 않았으며,
  - batchsize가 늘어나면 이대로 메모리 사용량이 늘어날 것. 
  - GPU-util을 보면 GPU를 제대로 활용하지 못하는 것을 확인함.

## 2.2. Custom으로 DataParallel 사용하기

- DataParallel을 사용하면서 메모리 불균형 문제를 해결할 수 있는 방법에 대한 힌트
- GPU 메모리가 늘어나는 것은 <span class="mark">모델의 출력을 하나의 GPU로 모은 것이기 때문</span>이며,
- 왜 하나의 GPU로 출력을 모으냐면, <span class="mark">모델의 출력을 사용해서 loss function을 계산</span>해야하기 때문.
  - 모델은 DataParallel로 병렬로 연산을 하게 했지만,
  - loss function이 그대로이기 떄문에 하나의 GPU에서 loss를 계산함.
  - <span class="mark">loss function도 Parallel하게 만들어서, 병렬로 연산하도록 만든다면~!</span>
  
![image](https://user-images.githubusercontent.com/48466625/63991721-daffc580-cb23-11e9-9781-dcef6f960b99.png)

- ```DataParallelCriterion```을 사용할 경우 일반적인 DataParallel 모델로 감싸면 안됨.
  - DataParallel은 기본적으로 하나의 GPU로 출력을 모으기 떄문,
- <span class="mark">DataParallelModel과 DataParallelCriterion을 사용하자.</span>

In [ ]:
import torch
import torch.nn as nn
from parallel import DataParallelModel, DataParallelCriterion

model = BERT(args)
model = DataParallelModel(model)
model.cuda()

criterion = nn.NLLLoss()
criterion = DataParallelCriterion(criterion) 

...

for i, (inputs, labels) in enumerate(trainloader):
    outputs = model(inputs)          
    loss = criterion(outputs, labels)     
    
    optimizer.zero_grad()
    loss.backward()                        
    optimizer.step()

![image](https://user-images.githubusercontent.com/48466625/63991842-42b61080-cb24-11e9-971e-2cbe5ab10753.png)

- DataParallel만 사용할 때보다, 1번과 2번 GPU의 메모리 차이가 상당히 줄었음.
- <span class="mark">하지만, GPU-Util을 보면 여전히 제대로 활용을 못하고 있음.</span>

## 2.3. Distributed 패키지 사용

- 분산학습은 여러 컴퓨터(머신)를 사용해서 학습되기 위한 것이지만, 
- <span class="mark">분산학습을 통해 하나의 머신에서 여러 GPU로 학습도 가능함.</span>

In [ ]:
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel


def main():
    args = parser.parse_args()

    ngpus_per_node = torch.cuda.device_count()
    args.world_size = ngpus_per_node * args.world_size
    mp.spawn(main_worker, nprocs=ngpus_per_node, 
             args=(ngpus_per_node, args))
    
    
def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu
    torch.cuda.set_device(args.gpu)
    
    print("Use GPU: {} for training".format(args.gpu))
    args.rank = args.rank * ngpus_per_node + gpu
    
    # 각 GPU 마다 분산 학습을 위한 초기화를 실행
    # multi-GPU의 경우 backend로 'nccl'을 사용
    # init_method에서는 FREEPORT에 사용 가능한 port를 적으면 됨
    dist.init_process_group(backend='nccl', 
                            init_method='tcp://127.0.0.1:FREEPORT',
                            world_size=args.world_size, 
                            rank=args.rank)
    
    model = Bert()
    model.cuda(args.gpu)
    
    # DataParallel 대신에 DistributedDataParallel을 사용
    model = DistributedDataParallel(model, device_ids=[args.gpu])

    acc = 0
    for i in range(args.num_epochs):
        model = train(model)
        acc = test(model, acc)

- DataLoader가 각 인풋을 프로세스에 전달하기 위해서, <span class="mark">```DistriubtedSampler```를 사용함</span>
  - 이는 ```DistributedDataParallel```과 함께 사용해야 함.
- 사용 방법은 dataset을 DistributedSampler로 감싸주고, DataLoader에서 이를 sampler 인자에 넣어줌.

In [ ]:
from torch.utils.data.distributed import DistributedSampler

train_dataset = datasets.ImageFolder(traindir, ...)
train_sampler = DistributedSampler(train_dataset)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=True, sampler=train_sampler)

![image](https://user-images.githubusercontent.com/48466625/63993452-83fdee80-cb2b-11e9-8e64-45a06bd5d414.png)

- 메모리 사용량이 완전 동일하고, GPU-util 수치도 상당히 높다.
- 하지만, <span class="mark">분산학습시에 간간히 문제가 발생할 수 있음.</span>
  - 학습에 사용하지 않는 파라미터가 있을 경우 문제를 일으킨다 등등..
- 신경 안쓰고 학습하려면?
  - <span class="mark">Nvidia의 Apex 패키지?</span>

## 2.4. Nvidia Apex 패키지

- Nvidia Apex 패키지, mixed precision 연산을 위한 패키지.
  - 보통 32비트 연산을 하는 딥러닝을 <span class="mark">16비트 연산을 사용해 메모리를 절약하고 속도를 높이겠다는 것.</span>
  - Distributed 관련 기능이 포함됨.
  - apex에서 ```DistributedDataParallel```을 임포트해서 사용함.
  - 따로 멀티 프로세싱을 진행하지 않음

In [ ]:
import torch.distributed as dist
from apex.parallel import DistributedDataParallel as DDP


def main():
    global args
    
    args.gpu = 0
    args.world_size = 1
    
    args.gpu = args.local_rank
    torch.cuda.set_device(args.gpu)
    torch.distributed.init_process_group(backend='nccl',
                                         init_method='env://')
    args.world_size = torch.distributed.get_world_size()
    
    model = Bert()
    model.cuda(args.gpu)
    
    # 모델을 DDP로 감싸줌
    model = DDP(model, delay_allreduce=True)

    acc = 0
    for i in range(args.num_epochs):
        model = train(model)
        acc = test(model, acc)

In [ ]:
### 실행 방법

python -m torch.distributed.launch --nproc_per_node=4 main.py \
    --batch_size 60 \
    --num_workers 2 \
    --gpu_devices 0 1 2 3\
    --distributed \
    --log_freq 100 

![image](https://user-images.githubusercontent.com/48466625/63993838-439f7000-cb2d-11e9-9363-78d6ceed29ae.png)


# 3. 학습법 선택하기

- DataParallel -> 불균형 문제
- Custom DataParallel -> 불균형은 해결하지만 GPU-util 문제
- Distriubted DataParallel -> 불균형, GPU-util 해결하지만, 간간히 문제
- Nvidia Apex

그러나,항상 Apex가 좋은 것은 아님.

- 가령, 이미지 분류 학습시에는 DataParallel만으로도 충분함.
  - GPU 메모리 불균형 문제는 BERT 같은 알고리즘이 출력히 상당히 크기 때문.

# 4. Tutorials

- model = nn.DataParallel(model

## 4.1. Dataset

In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
 
# Parameters and DataLoaders
input_size = 5
output_size = 2
 
batch_size = 30
data_size = 100

In [12]:
class RandomDataset(Dataset):
    
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len

random_loader = DataLoader(dataset=RandomDataset(input_size, 100), batch_size=batch_size, shuffle=True)

## 4.2. Simple model

In [15]:
class Model(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("Our model: input_size", input.size(), "output_size", output.size())
        
        return output

## 4.3. DataParallel 

In [13]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model, device_ids=[0,2])

if torch.cuda.is_available():
    model.cuda()

Let's use 8 GPUs!


## 4.4. Run the model


In [14]:
for data in random_loader:
    if torch.cuda.is_available():
        input_var = Variable(data.cuda())
    else:
        input_var = Variable(data)
    
    output = model(input_var)
    print("Outside: input size", input_var.size(), "output size", output.size())

Our model: input_size torch.Size([15, 5]) output_size torch.Size([15, 2])
Our model: input_size torch.Size([15, 5]) output_size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Our model: input_size torch.Size([15, 5]) output_size torch.Size([15, 2])
Our model: input_size torch.Size([15, 5]) output_size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Our model: input_size torch.Size([15, 5]) output_size torch.Size([15, 2])
Our model: input_size torch.Size([15, 5]) output_size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Our model: input_size torch.Size([5, 5]) output_size torch.Size([5, 2])
Our model: input_size torch.Size([5, 5]) output_size torch.Size([5, 2])
Outside: input size torch.Size([10, 5]) output size torch.Size([10, 2])
